In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve

import libsbml
from process_bigraph import pp 

from test_suite.test_data_model import TestCompositionResults

In [2]:
test_archive_fp = '../test_suite/examples/sbml-core/Varusai-Sci-Rep-2018-mTOR-signaling-LSODA-LSODAR-SBML.omex'
os.path.exists(test_archive_fp)

True

In [39]:
from dataclasses import dataclass
from typing import *
from tempfile import mkdtemp

import requests
from process_bigraph import pp, pf 
from biosimulators_utils.combine.io import CombineArchiveReader
from biosimulators_utils.combine.data_model import CombineArchive

from biosimulator_processes.data_model import _BaseClass


@dataclass 
class ProjectsQuery(_BaseClass):
    project_ids: List[str]
    project_data: Dict


@dataclass
class ArchiveFiles(_BaseClass):
    run_id: str 
    project_name: str 
    files: List[Dict]

    
async def search_projects_archives(query: str) -> ProjectsQuery:
    get_all_projects_url = 'https://api.biosimulations.dev/projects'
    headers = {'accept': 'application/json'}
    try:
        all_projects_resp = requests.get(get_all_projects_url, headers=headers)
        all_projects_resp.raise_for_status()
        all_projects = all_projects_resp.json()
        project_ids = [p['id'].lower() for p in all_projects]
        
        query_result_ids = [project_id for project_id in project_ids if query.lower() in project_id]
        query_result_data = {}
        for project in all_projects:
            project_id = project['id'].lower()
            if project_id in query_result_ids:
                query_result_data[project_id] = project
        
        return ProjectsQuery(
            project_ids=query_result_ids,
            project_data=query_result_data)
    except Exception as e:
        print(f'Failed to fetch OMEX archive:\n{e}')
        

async def get_run_id_files(run_id: str, project_name: str) -> ArchiveFiles:
    get_files_url = f'https://api.biosimulations.dev/files/{run_id}'
    headers = {'accept': 'application/json'}
    try:
        files_resp = requests.get(get_files_url, headers=headers)
        run_files = files_resp.json()
        return ArchiveFiles(run_id=run_id, project_name=project_name, files=run_files)
    except Exception as e:
        print(f'Failed to fetch simulation run files:\n{e}')


async def fetch_archive_files(query: str) -> ArchiveFiles:
    archive_query = await search_projects_archives(query)
    query_results = archive_query.project_ids
    
    options_menu = dict(zip(list(range(len(query_results))), query_results))

    user_selection = int(input(f'Please enter your archive selection:\n{pf(options_menu)}'))
    
    selected_project_id = options_menu[user_selection]
    simulation_run_id = archive_query.project_data[selected_project_id]['simulationRun']
    simulation_run_files = await get_run_id_files(simulation_run_id, selected_project_id)
    return simulation_run_files

    
async def extract_model_from_query(query: str, save_dir=None):
    archive_files = await fetch_archive_files(query)
    get_file_url = f'https://api.biosimulations.dev/files/{archive_files.run_id}'
    
    model_file_location = [file['location'] for file in archive_files.files if file['location'].endswith('.xml')].pop()
    model_file_resp = requests.get(get_file_url + f'/{model_file_location}/download', headers={'accept': 'application/xml'})
    
    model_fp = os.path.join(save_dir or mkdtemp(), model_file_location)
    with open(model_fp, 'w') as f:
        f.write(model_file_resp.text)
    
    pp(model_file_resp.text)
            
    

In [40]:
await extract_model_from_query(query='Yeast')

("<?xml version='1.0' encoding='UTF-8' standalone='no'?>\n"
 '<sbml xmlns="http://www.sbml.org/sbml/level3/version1/core" '
 'layout:required="false" level="3" qual:required="true" '
 'xmlns:layout="http://www.sbml.org/sbml/level3/version1/layout/version1" '
 'version="1" '
 'xmlns:qual="http://www.sbml.org/sbml/level3/version1/qual/version1">\n'
 '  <model id="model_id">\n'
 '    <layout:listOfLayouts '
 'xmlns:layout="http://www.sbml.org/sbml/level3/version1/layout/version1" '
 'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n'
 '      <layout:layout layout:id="__layout__">\n'
 '        <layout:dimensions layout:height="519" layout:width="614"/>\n'
 '        <layout:listOfAdditionalGraphicalObjects>\n'
 '          <layout:generalGlyph layout:id="_ly_Cln3" '
 'layout:reference="Cln3">\n'
 '            <layout:boundingBox>\n'
 '              <layout:position layout:x="403" layout:y="58"/>\n'
 '              <layout:dimensions layout:height="30" layout:width="60"/>\n'
 '        